In [1]:
import pandas as pd
from nltk.util import ngrams

import numpy as np
import os 
import unicodedata

In [2]:
# Reading in the Turkish nams 
df_turk  =  pd.read_excel('name_data/exigerData/EXGR_Turkish names2.xlsx')
print(df_turk)

                                            id                      label_tr  \
0       http://www.wikidata.org/entity/Q282782                          Ceza   
1       http://www.wikidata.org/entity/Q288019                        Gülşen   
2       http://www.wikidata.org/entity/Q236047                        Hadise   
3       http://www.wikidata.org/entity/Q236920                         Emrah   
4       http://www.wikidata.org/entity/Q182948                        Göksel   
...                                        ...                           ...   
18769  http://www.wikidata.org/entity/Q6053953     Elif Nur Bozkurt Tandoğan   
18770  http://www.wikidata.org/entity/Q6085044    Muhammed Ali Fatih Erbakan   
18771  http://www.wikidata.org/entity/Q6041731      İsmail Hamit Özer Derbil   
18772  http://www.wikidata.org/entity/Q6071196        Vedat Ali Özkan Kayacı   
18773     http://www.wikidata.org/entity/Q5031  Elder Paisios of Mount Athos   

      comment          Given name Middl

In [3]:
#printing out the shape 
df_turk.shape

(18774, 8)

In [4]:
#Printing out the head
df_turk.head()

,id,label_tr,comment,Given name,Middle Name,Family name,Maiden Name,Unnamed: 7
0,http://www.wikidata.org/entity/Q282782,Ceza,x,NaN,NaN,NaN,NaN,1
1,http://www.wikidata.org/entity/Q288019,Gülşen,x,NaN,NaN,NaN,NaN,1
2,http://www.wikidata.org/entity/Q236047,Hadise,x,NaN,NaN,NaN,NaN,1
3,http://www.wikidata.org/entity/Q236920,Emrah,x,NaN,NaN,NaN,NaN,1
4,http://www.wikidata.org/entity/Q182948,Göksel,x,NaN,NaN,NaN,NaN,1


In [5]:
#checking if there is any null entries in the datasets
df_turk.isnull().values.any()

True

In [6]:
# checking what's in the comment columns
df_turk['comment']

0          x
1          x
2          x
3          x
4          x
        ... 
18769    NaN
18770    NaN
18771      x
18772    NaN
18773      x
Name: comment, Length: 18774, dtype: object

In [7]:
df_turk.isnull().head()

,id,label_tr,comment,Given name,Middle Name,Family name,Maiden Name,Unnamed: 7
0,False,False,False,True,True,True,True,False
1,False,False,False,True,True,True,True,False
2,False,False,False,True,True,True,True,False
3,False,False,False,True,True,True,True,False
4,False,False,False,True,True,True,True,False


In [8]:
#checking how many null entries each columns have 
nan_count = np.sum(df_turk.isnull(), axis = 0)
nan_count

id                 0
label_tr           0
comment        18318
Given name       455
Middle Name    18772
Family name      459
Maiden Name    18763
Unnamed: 7         0
dtype: int64

In [9]:
# checking if the datasets have any duplicates 
df_turk['label_tr'].duplicated().any()

True

In [10]:
#check if there is any null entries 
np.any(df_turk['label_tr'].isnull())

False

In [12]:
#just checking the shape before dropping dupliates (18774,8)
df_turk.shape

(18774, 8)

In [13]:
#Dropping the duplicates(place in a new variable)
df2_turk = df_turk.drop_duplicates(subset=['label_tr'])
print(df2_turk)

                                            id                      label_tr  \
0       http://www.wikidata.org/entity/Q282782                          Ceza   
1       http://www.wikidata.org/entity/Q288019                        Gülşen   
2       http://www.wikidata.org/entity/Q236047                        Hadise   
3       http://www.wikidata.org/entity/Q236920                         Emrah   
4       http://www.wikidata.org/entity/Q182948                        Göksel   
...                                        ...                           ...   
18769  http://www.wikidata.org/entity/Q6053953     Elif Nur Bozkurt Tandoğan   
18770  http://www.wikidata.org/entity/Q6085044    Muhammed Ali Fatih Erbakan   
18771  http://www.wikidata.org/entity/Q6041731      İsmail Hamit Özer Derbil   
18772  http://www.wikidata.org/entity/Q6071196        Vedat Ali Özkan Kayacı   
18773     http://www.wikidata.org/entity/Q5031  Elder Paisios of Mount Athos   

      comment          Given name Middl

In [14]:
#checking if there are still duplicates
df2_turk['label_tr'].duplicated().any()


False

In [15]:
#Just checking if th shape went down(which it did (18774,8) to (18491,8))
df2_turk.shape

(18491, 8)

In [16]:
# checking if the null values went down (it did) 
nan_count = np.sum(df2_turk.isnull(), axis = 0)
nan_count

id                 0
label_tr           0
comment        18037
Given name       453
Middle Name    18489
Family name      457
Maiden Name    18480
Unnamed: 7         0
dtype: int64

In [17]:
#printing the col_names with null values
condition = nan_count != 0
col_names = nan_count[condition].index
nan_cols = list(col_names)
print(nan_cols)

['comment', 'Given name', 'Middle Name', 'Family name', 'Maiden Name']


In [18]:
print(df2_turk['comment'].unique())
print(df2_turk['Given name'].unique())
print(df2_turk['Middle Name'].unique())
print(df2_turk['Family name'].unique())
print(df2_turk['Maiden Name'].unique())

['x' nan]
[nan 'Melahat' 'Freddy' ... 'Elif Nur Bozkurt ' 'Muhammed Ali Fatih'
 'Vedat Ali Özkan']
[nan 'Sahin' 'Kemal']
[nan 'Abbasova' 'Scholl' ... 'Kapıcıoğlu' 'Ali Marandi' 'Bektur']
[nan 'Önder' 'Kafaoğlu' 'Türker' 'Doğan' 'Karahan' 'Sabancı' 'Ekşi' 'İnan'
 'Tosun' 'Sayan']


In [19]:
# drop comments row with x 
comments_drop_with_x =  df2_turk['comment'] == 'x'
df2_turk = df2_turk.drop(df2_turk[df2_turk['comment'] == 'x'].index)
print(df2_turk)
#print(comments_drop)
#df2_turk['comment']

                                             id                    label_tr  \
125     http://www.wikidata.org/entity/Q3508136            Melahat Abbasova   
126       http://www.wikidata.org/entity/Q96602         Freddy Sahin-Scholl   
127      http://www.wikidata.org/entity/Q217097             Richard Kingson   
129      http://www.wikidata.org/entity/Q110126                  Hrant Dink   
130       http://www.wikidata.org/entity/Q42079               Ricky Winslow   
...                                         ...                         ...   
18764  http://www.wikidata.org/entity/Q49703809   Nil İpek Hülagü Öztürkmen   
18765  http://www.wikidata.org/entity/Q24230049      Fatma Betül Sayan Kaya   
18769   http://www.wikidata.org/entity/Q6053953   Elif Nur Bozkurt Tandoğan   
18770   http://www.wikidata.org/entity/Q6085044  Muhammed Ali Fatih Erbakan   
18772   http://www.wikidata.org/entity/Q6071196      Vedat Ali Özkan Kayacı   

      comment          Given name Middle Name Famil

In [20]:
#check the shape again( went form (18491,8) to (18037, 8))
df2_turk.shape

(18037, 8)

In [21]:
nan_count = np.sum(df2_turk.isnull(), axis = 0)
nan_count

id                 0
label_tr           0
comment        18037
Given name         0
Middle Name    18035
Family name        4
Maiden Name    18026
Unnamed: 7         0
dtype: int64

In [22]:
non_alnum_names_turk = [name for name in df2_turk['label_tr'] if not str.isalnum(name.replace(' ', ''))]
print(len(non_alnum_names_turk))
non_alnum_names_turk

30


['Freddy Sahin-Scholl',
 'Perihan Önder-Ridder',
 'Erhan-Can Kartal',
 'Ali-Özgür Özdil',
 'Mehmet Aga-Oglu',
 'Ahmed Agha-Oghlu',
 'Hatice Aksoy-Woinek',
 'Bugha al-Sharabi',
 'Iffat Al-Thunayan',
 "Nev'îzâde Atâyî",
 'Üstün Bilgen-Reinart',
 'Mehpare Bozyigit-Kirchmann',
 'Elif Çağlar-Muslu',
 "Cem'i Demiroğlu",
 "Neş'e Erdok",
 'Asuman Kafaoğlu-Büke',
 'Elçin Kürşat-Ahlers',
 'Gönül Sen-Menzel',
 'Tülay Sözbir-Seidel',
 "Gaybi Sun'ullah",
 'Ayshe Talay-Ongan',
 'Sevgi Türker-Terlemez',
 "Mümtaz'er Türköne",
 'Sabiha Bânu Yalkut-Breddermann',
 "Temel Nücûm'i Göksel",
 'Molla Ahmed-i Cezirî',
 "Dipika O'Neill Joti",
 "Abdurrahman Necati Kara'a",
 'Kerimüddin Mahmud-i Aksarayî',
 'Salah al-Din Zarkub']

In [23]:
#drop extra column we don't need 
df2_turk = df2_turk.drop(columns = ['Unnamed: 7'])

In [24]:
nan_count = np.sum(df2_turk.isnull(), axis = 0)
nan_count

id                 0
label_tr           0
comment        18037
Given name         0
Middle Name    18035
Family name        4
Maiden Name    18026
dtype: int64

In [26]:
#Drop columns that won't be used in df2
print(df2_turk.columns)
df2_turk = df2_turk.drop(columns= [ 'comment', 'Given name', 'Middle Name', 'Family name',
       'Maiden Name'])


Index(['id', 'label_tr', 'comment', 'Given name', 'Middle Name', 'Family name',
       'Maiden Name'],
      dtype='object')


Now For Feature Engineering 

In [30]:
#creating features to determine the alphabet
determine_alphabet = [unicodedata.name(name[0]).split(' ')[0] for name in df2_turk['label_tr']]
#df2_turk['determine_alphabet'] = df2_turk['label_tr'].apply(lambda name: unicodedata.name(name[0]).split(' ')[0])
df2_turk['determine_alphabet'] = determine_alphabet


In [31]:
print(df2_turk['determine_alphabet'])

125      LATIN
126      LATIN
127      LATIN
129      LATIN
130      LATIN
         ...  
18764    LATIN
18765    LATIN
18769    LATIN
18770    LATIN
18772    LATIN
Name: determine_alphabet, Length: 18037, dtype: object


In [34]:
#char grams
def get_ngrams(text, n):
    if isinstance(text,str):
        name = list(text)
    ngrams_list =  list(ngrams(list(text), n))
    return ngrams_list

df2_turk["unigrams"] = df2_turk['label_tr'].apply(lambda name: list(name) if isinstance(name, str) else [])
df2_turk["bigrams"] = df2_turk['label_tr'].apply(lambda name: get_ngrams(name,2) if isinstance(name, str) else [])
df2_turk["trigrams"] = df2_turk['label_tr'].apply(lambda name: get_ngrams(name,3) if isinstance(name, str) else [])

df2_turk['char_ngrams'] = df2_turk["unigrams"] + df2_turk["bigrams"] + df2_turk["trigrams"]

In [35]:
print(df2_turk['char_ngrams'])

125      [M, e, l, a, h, a, t,  , A, b, b, a, s, o, v, ...
126      [F, r, e, d, d, y,  , S, a, h, i, n, -, S, c, ...
127      [R, i, c, h, a, r, d,  , K, i, n, g, s, o, n, ...
129      [H, r, a, n, t,  , D, i, n, k, (H, r), (r, a),...
130      [R, i, c, k, y,  , W, i, n, s, l, o, w, (R, i)...
                               ...                        
18764    [N, i, l,  , İ, p, e, k,  , H, ü, l, a, g, ü, ...
18765    [F, a, t, m, a,  , B, e, t, ü, l,  , S, a, y, ...
18769    [E, l, i, f,  , N, u, r,  , B, o, z, k, u, r, ...
18770    [M, u, h, a, m, m, e, d,  , A, l, i,  , F, a, ...
18772    [V, e, d, a, t,  , A, l, i,  , Ö, z, k, a, n, ...
Name: char_ngrams, Length: 18037, dtype: object


In [36]:
# feature to check the name length
def name_length(name):
    return len(name.replace(' ', ''))

df2_turk['name_len'] = df2_turk['label_tr'].apply(name_length)

In [37]:
print(df2_turk['name_len'])

125      15
126      18
127      14
129       9
130      12
         ..
18764    22
18765    19
18769    22
18770    23
18772    19
Name: name_len, Length: 18037, dtype: int64


In [38]:
# feature to check the name length
def token_length(name):
    return len(name.split())

df2_turk['token_len'] = df2_turk['label_tr'].apply(token_length)

In [39]:
print(df2_turk['token_len'])


125      2
126      2
127      2
129      2
130      2
        ..
18764    4
18765    4
18769    4
18770    4
18772    4
Name: token_len, Length: 18037, dtype: int64


In [42]:
df2_turk['period_freq'] = df2_turk['label_tr'].apply(lambda name: name.count('.'))
df2_turk['dash_freq'] = df2_turk['label_tr'].apply(lambda name: name.count('-'))
df2_turk['space_freq'] = df2_turk['label_tr'].apply(lambda name: name.count(' '))

In [43]:
df2_turk

,id,label_tr,determine_alphabet,unigrams,bigrams,trigrams,char_ngrams,name_len,token_len,period_freq,dash_freq,space_freq
125,http://www.wikidata.org/entity/Q3508136,Melahat Abbasova,LATIN,"[M, e, l, a, h, a, t, , A, b, b, a, s, o, v, a]","[(M, e), (e, l), (l, a), (a, h), (h, a), (a, t...","[(M, e, l), (e, l, a), (l, a, h), (a, h, a), (...","[M, e, l, a, h, a, t, , A, b, b, a, s, o, v, ...",15,2,0,0,1
126,http://www.wikidata.org/entity/Q96602,Freddy Sahin-Scholl,LATIN,"[F, r, e, d, d, y, , S, a, h, i, n, -, S, c, ...","[(F, r), (r, e), (e, d), (d, d), (d, y), (y, ...","[(F, r, e), (r, e, d), (e, d, d), (d, d, y), (...","[F, r, e, d, d, y, , S, a, h, i, n, -, S, c, ...",18,2,0,1,1
127,http://www.wikidata.org/entity/Q217097,Richard Kingson,LATIN,"[R, i, c, h, a, r, d, , K, i, n, g, s, o, n]","[(R, i), (i, c), (c, h), (h, a), (a, r), (r, d...","[(R, i, c), (i, c, h), (c, h, a), (h, a, r), (...","[R, i, c, h, a, r, d, , K, i, n, g, s, o, n, ...",14,2,0,0,1
129,http://www.wikidata.org/entity/Q110126,Hrant Dink,LATIN,"[H, r, a, n, t, , D, i, n, k]","[(H, r), (r, a), (a, n), (n, t), (t, ), ( , D...","[(H, r, a), (r, a, n), (a, n, t), (n, t, ), (...","[H, r, a, n, t, , D, i, n, k, (H, r), (r, a),...",9,2,0,0,1
130,http://www.wikidata.org/entity/Q42079,Ricky Winslow,LATIN,"[R, i, c, k, y, , W, i, n, s, l, o, w]","[(R, i), (i, c), (c, k), (k, y), (y, ), ( , W...","[(R, i, c), (i, c, k), (c, k, y), (k, y, ), (...","[R, i, c, k, y, , W, i, n, s, l, o, w, (R, i)...",12,2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
18764,http://www.wikidata.org/entity/Q49703809,Nil İpek Hülagü Öztürkmen,LATIN,"[N, i, l, , İ, p, e, k, , H, ü, l, a, g, ü, ...","[(N, i), (i, l), (l, ), ( , İ), (İ, p), (p, e...","[(N, i, l), (i, l, ), (l, , İ), ( , İ, p), (...","[N, i, l, , İ, p, e, k, , H, ü, l, a, g, ü, ...",22,4,0,0,3
18765,http://www.wikidata.org/entity/Q24230049,Fatma Betül Sayan Kaya,LATIN,"[F, a, t, m, a, , B, e, t, ü, l, , S, a, y, ...","[(F, a), (a, t), (t, m), (m, a), (a, ), ( , B...","[(F, a, t), (a, t, m), (t, m, a), (m, a, ), (...","[F, a, t, m, a, , B, e, t, ü, l, , S, a, y, ...",19,4,0,0,3
18769,http://www.wikidata.org/entity/Q6053953,Elif Nur Bozkurt Tandoğan,LATIN,"[E, l, i, f, , N, u, r, , B, o, z, k, u, r, ...","[(E, l), (l, i), (i, f), (f, ), ( , N), (N, u...","[(E, l, i), (l, i, f), (i, f, ), (f, , N), (...","[E, l, i, f, , N, u, r, , B, o, z, k, u, r, ...",22,4,0,0,3
18770,http://www.wikidata.org/entity/Q6085044,Muhammed Ali Fatih Erbakan,LATIN,"[M, u, h, a, m, m, e, d, , A, l, i, , F, a, ...","[(M, u), (u, h), (h, a), (a, m), (m, m), (m, e...","[(M, u, h), (u, h, a), (h, a, m), (a, m, m), (...","[M, u, h, a, m, m, e, d, , A, l, i, , F, a, ...",23,4,0,0,3
